# Garmin Tests Report

## This notebook is for tests only

In [1]:
import tcxtools
import graph
import helper
import correlation
from math import radians, cos, sin, asin, sqrt
from geopy import distance
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6373 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# TCX and CSV files name without extension

In [3]:
filename = 'activity_5173186556'

# Reading .TCX file

In [4]:
tcx_data = tcxtools.TCXPandas('./cycling/' + filename + '.tcx')
tracks, laps = tcx_data.parse()
#tracks = tracks.set_index('time')

# Tracking points

In [5]:
#tracks.dropna(inplace=True)
tracks.rename(columns={'distance':'distance cumulative'}, inplace=True)
tracks['GARMIN'] = tracks['distance cumulative'].diff()
#tracks

```
               model             major (km)   minor (km)     flattening
ELLIPSOIDS = {'WGS-84':        (6378.137,    6356.7523142,  1 / 298.257223563),
              'GRS-80':        (6378.137,    6356.7523141,  1 / 298.257222101),
              'Airy (1830)':   (6377.563396, 6356.256909,   1 / 299.3249646),
              'Intl 1924':     (6378.388,    6356.911946,   1 / 297.0),
              'Clarke (1880)': (6378.249145, 6356.51486955, 1 / 293.465),
              'GRS-67':        (6378.1600,   6356.774719,   1 / 298.25),
              }
```

In [6]:
df = pd.DataFrame(tracks['GARMIN'])

for i in range(1, len(tracks)):

    if pd.notna(tracks.loc[i, 'latitude']) and pd.notna(tracks.loc[i-1, 'latitude']):
        
        p1 = (tracks.loc[i-1, 'latitude'], tracks.loc[i-1, 'longitude'], tracks.loc[i-1, 'altitude'])
        p2 = (tracks.loc[i, 'latitude'], tracks.loc[i, 'longitude'], tracks.loc[i, 'altitude'])
        
        df.loc[i, 'WGS-84'] = distance.geodesic(p1[:2], p2[:2], ellipsoid='WGS-84').m
        df.loc[i, 'GRS-80'] = distance.geodesic(p1[:2], p2[:2], ellipsoid='GRS-80').m
        df.loc[i, 'Airy (1830)'] = distance.geodesic(p1[:2], p2[:2], ellipsoid='Airy (1830)').m
        df.loc[i, 'Intl 1924'] = distance.geodesic(p1[:2], p2[:2], ellipsoid='Intl 1924').m
        df.loc[i, 'Clarke (1880)'] = distance.geodesic(p1[:2], p2[:2], ellipsoid='Clarke (1880)').m
        df.loc[i, 'GRS-67'] = distance.geodesic(p1[:2], p2[:2], ellipsoid='GRS-67').m
        df.loc[i, 'Great Circle'] = distance.great_circle(p1[:2], p2[:2]).m
        df.loc[i, 'Haversine'] = haversine(p1[1], p1[0], p2[1], p2[0]) * 1000
        
    else:

        df.loc[i, 'WGS-84'] = np.nan
        df.loc[i, 'GRS-80'] = np.nan
        df.loc[i, 'Airy (1830)'] = np.nan
        df.loc[i, 'Intl 1924'] = np.nan
        df.loc[i, 'Clarke (1880)'] = np.nan
        df.loc[i, 'GRS-67'] = np.nan
        df.loc[i, 'Great Circle'] = np.nan
        df.loc[i, 'Haversine'] = np.nan

In [7]:
df_al = pd.DataFrame(tracks['GARMIN'])

for i in range(1, len(df_al)):

    if pd.notna(tracks.loc[i, 'latitude']) and pd.notna(tracks.loc[i-1, 'latitude']):
        
        p1 = (tracks.loc[i-1, 'latitude'], tracks.loc[i-1, 'longitude'], tracks.loc[i-1, 'altitude'])
        p2 = (tracks.loc[i, 'latitude'], tracks.loc[i, 'longitude'], tracks.loc[i, 'altitude'])
        
        # euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
        
        df_al.loc[i, 'WGS-84'] = sqrt( df.loc[i, 'WGS-84']**2 + (p2[2] - p1[2])**2 )
        df_al.loc[i, 'GRS-80'] = sqrt( df.loc[i, 'GRS-80']**2 + (p2[2] - p1[2])**2 )
        df_al.loc[i, 'Airy (1830)'] = sqrt( df.loc[i, 'Airy (1830)']**2 + (p2[2] - p1[2])**2 )
        df_al.loc[i, 'Intl 1924'] = sqrt( df.loc[i, 'Intl 1924']**2 + (p2[2] - p1[2])**2 )
        df_al.loc[i, 'Clarke (1880)'] = sqrt( df.loc[i, 'Clarke (1880)']**2 + (p2[2] - p1[2])**2 )
        df_al.loc[i, 'GRS-67'] = sqrt( df.loc[i, 'GRS-67']**2 + (p2[2] - p1[2])**2 )
        df_al.loc[i, 'Great Circle'] = sqrt( df.loc[i, 'Great Circle']**2 + (p2[2] - p1[2])**2 )
        df_al.loc[i, 'Haversine'] = sqrt( df.loc[i, 'Haversine']**2 + (p2[2] - p1[2])**2 )

    else:

        df.loc[i, 'WGS-84'] = np.nan
        df.loc[i, 'GRS-80'] = np.nan
        df.loc[i, 'Airy (1830)'] = np.nan
        df.loc[i, 'Intl 1924'] = np.nan
        df.loc[i, 'Clarke (1880)'] = np.nan
        df.loc[i, 'GRS-67'] = np.nan
        df.loc[i, 'Great Circle'] = np.nan

# Distance desconsidering altitude

In [8]:
df.dropna(inplace=True)
df

,GARMIN,WGS-84,GRS-80,Airy (1830),Intl 1924,Clarke (1880),GRS-67,Great Circle,Haversine
9,9.250000,9.597356,9.597356,9.596569,9.597643,9.597176,9.597390,9.608021,9.611024
10,3.590000,4.028972,4.028972,4.028645,4.029087,4.028877,4.028986,4.034594,4.035855
11,2.570000,5.742063,5.742063,5.741654,5.742161,5.741671,5.742083,5.765839,5.767641
12,2.470001,3.118863,3.118863,3.118604,3.118960,3.118820,3.118874,3.121419,3.122394
13,11.250000,13.601789,13.601789,13.600719,13.602142,13.601324,13.601837,13.629731,13.633991
...,...,...,...,...,...,...,...,...,...
1069,16.460938,16.488025,16.488025,16.486773,16.488400,16.487259,16.488083,16.534369,16.539536
1070,19.410156,19.450026,19.450026,19.448583,19.450427,19.448964,19.450094,19.514277,19.520375
1071,1.779297,2.617641,2.617641,2.617409,2.617741,2.617674,2.617651,2.615547,2.616365
1072,0.480469,3.009968,3.009968,3.009720,3.010059,3.009917,3.009978,3.012989,3.013931


# Distance considering altitude

In [9]:
df_al.dropna(inplace=True)
df_al

,GARMIN,WGS-84,GRS-80,Airy (1830),Intl 1924,Clarke (1880),GRS-67,Great Circle,Haversine
9,9.250000,9.597356,9.597356,9.596569,9.597643,9.597176,9.597390,9.608021,9.611024
10,3.590000,4.028972,4.028972,4.028645,4.029087,4.028877,4.028986,4.034594,4.035855
11,2.570000,5.745543,5.745543,5.745135,5.745641,5.745151,5.745563,5.769305,5.771105
12,2.470001,3.118863,3.118863,3.118604,3.118960,3.118820,3.118874,3.121419,3.122394
13,11.250000,13.603260,13.603260,13.602191,13.603613,13.602796,13.603308,13.631200,13.635459
...,...,...,...,...,...,...,...,...,...
1069,16.460938,16.492877,16.492877,16.491626,16.493252,16.492111,16.492935,16.539207,16.544373
1070,19.410156,19.454139,19.454139,19.452697,19.454540,19.453078,19.454207,19.518377,19.524474
1071,1.779297,2.617641,2.617641,2.617409,2.617741,2.617674,2.617651,2.615547,2.616365
1072,0.480469,3.009968,3.009968,3.009720,3.010059,3.009917,3.009978,3.012989,3.013931


In [10]:
s1 = df.sum()

In [11]:
df1 = pd.DataFrame(s1)
df1.rename(columns={0:'total distance'}, inplace=True)
df1['diff w/o altitude'] = df1['total distance'] - df1['total distance'][0]

df1

,total distance,diff w/o altitude
GARMIN,19028.790234,0.000000
WGS-84,19037.702024,8.911790
GRS-80,19037.702023,8.911790
Airy (1830),19036.203706,7.413472
Intl 1924,19038.197480,9.407246
Clarke (1880),19037.057962,8.267728
GRS-67,19037.769223,8.978990
Great Circle,19076.394266,47.604033
Haversine,19082.355818,53.565585


In [12]:
df1.describe()

,total distance,diff w/o altitude
count,9.000000,9.000000
mean,19045.796971,17.006737
std,19.312002,19.312002
min,19028.790234,0.000000
25%,19037.057962,8.267728
50%,19037.702024,8.911790
75%,19038.197480,9.407246
max,19082.355818,53.565585


In [13]:
s2 = df_al.sum()

In [14]:
df2 = pd.DataFrame(s2)
df2.rename(columns={0:'total distance'}, inplace=True)
df2['diff w/ altitude'] = df2['total distance'] - df2['total distance'][0]

df2

,total distance,diff w/ altitude
GARMIN,19028.790234,0.000000
WGS-84,19051.871362,23.081129
GRS-80,19051.871362,23.081128
Airy (1830),19050.374182,21.583949
Intl 1924,19052.366421,23.576188
Clarke (1880),19051.227667,22.437433
GRS-67,19051.938512,23.148278
Great Circle,19090.541833,61.751600
Haversine,19096.498973,67.708740
